## *Pasos del ciclo de vida*

1. Carga de datos
2. Ingeneria de características
3. Escalado de características
4. Busqueda de optimización de parámetros
5. Entenamiento K-Means
6. Análisis y visialización
7. Guardado del modelo
8. Predicción


In [1]:
#Librerias generales
import pandas as pd
import numpy as nup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# librerias para guardado del modelo:
import joblib
from datetime import datetime
import warnings


In [2]:
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_excel("./Datasets/BD_Clientes_Productos.xlsx")
df = pd.DataFrame(data)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299169 entries, 0 to 299168
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Fecha           299169 non-null  datetime64[ns]
 1   Nombre Cliente  299169 non-null  object        
 2   Tipo Cliente    299169 non-null  object        
 3   Departamento    299169 non-null  object        
 4   Vendedor        299169 non-null  object        
 5   Sucursal        299169 non-null  object        
 6   Categoría       299169 non-null  object        
 7   Producto        299169 non-null  object        
 8   Linea           299169 non-null  object        
 9   Cantidad        299169 non-null  int64         
 10  Venta           299169 non-null  float64       
 11  Costos          299169 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 27.4+ MB


,Fecha,Nombre Cliente,Tipo Cliente,Departamento,Vendedor,Sucursal,Categoría,Producto,Linea,Cantidad,Venta,Costos
0,2018-01-04,OCEANO AZUL DISTRIBUCIONES S.A.S.,DISTRIBUIDOR,ATLANTICO,SILVIA CAROLINA LOPEZ,NORTE,LACTEA,CHEDDAR,QUESOS,5,45107.494665,41250.000000
1,2018-01-04,COMPAÑIA NACIONAL DE LEVADURAS LEVAPAN S.A.,CADENA REGIONAL,ANTIOQUIA,PAULA ANDREA ARTEAGA,CENTRO,LACTEA,MANTEQUILLA MINI,ESPARCIBLES,1,5717.000000,3712.872135
2,2018-01-04,RESTAURANTE JIANBANQ MEI,CADENA REGIONAL,ANTIOQUIA,PAULA ANDREA ARTEAGA,CENTRO,LACTEA,MANTEQUILLA MINI,ESPARCIBLES,1,5717.000000,3712.872135
3,2018-01-04,COMERCIALIZADORA Y ASOCIADOS S.A.S,CADENA REGIONAL,ANTIOQUIA,VICKY BUENAVENTURA,CENTRO,LACTEA,AREQUIPE MINI,POSTRES Y DULCES,1,8183.000000,4537.624369
4,2018-01-04,RAPI MERCAR SA,CADENA REGIONAL,ANTIOQUIA,LINA CECILIA CARDONA,CENTRO,LACTEA,MIGUELUCHO PER,POSTRES Y DULCES,1,8756.000000,5041.475267


In [4]:
snapshot_date = df['Fecha'].max() + pd.Timedelta(days=1)
# Agrupaciones por clientes:
rfm_data = df.groupby('Nombre Cliente').agg({
    'Fecha': lambda date: (snapshot_date - date.max()).days, # Recencia , es decir las ventas más recientes
    'Nombre Cliente': 'count', # Frecuencia, es decir el número de compras
    'Venta': 'sum' # Monetario, es decir el total de compras
})

display(rfm_data)

,Fecha,Nombre Cliente,Venta
Nombre Cliente,,,
DISTRIBUIDORA PUNTO SEIS,2,68,1.417332e+06
5 ESQUINAS,46,50,9.394804e+05
61PRADO EUROPEAN GUESTHOUSE,2,209,3.135524e+06
ABARROTES ALFER,33,52,8.029471e+05
ABARROTES CUCUTA,39,54,7.615975e+05
...,...,...,...
ZEUS,72,11,1.812906e+05
ZONA LOGISTICA S.A.S,1,210,4.527208e+06
ZONA REFRESCANTE,72,11,1.090489e+05


In [5]:
# Renombrar Las columnas
rfm_data.rename(columns={
    'Fecha': 'Recencia',    
    'Nombre Cliente': 'Frecuencia',
    'Venta': 'Monetario'
}, inplace=True)
display(rfm_data)

,Recencia,Frecuencia,Monetario
Nombre Cliente,,,
DISTRIBUIDORA PUNTO SEIS,2,68,1.417332e+06
5 ESQUINAS,46,50,9.394804e+05
61PRADO EUROPEAN GUESTHOUSE,2,209,3.135524e+06
ABARROTES ALFER,33,52,8.029471e+05
ABARROTES CUCUTA,39,54,7.615975e+05
...,...,...,...
ZEUS,72,11,1.812906e+05
ZONA LOGISTICA S.A.S,1,210,4.527208e+06
ZONA REFRESCANTE,72,11,1.090489e+05


In [6]:
# Escaladpo de los datos
scaler = StandardScaler()
rfm_scaler = scaler.fit_transform(rfm_data)

rfm_scaler_df = pd.DataFrame(rfm_scaler, index=rfm_data.index, columns=rfm_data.columns) # DataFrame con los datos escalados
display(rfm_scaler_df) 


,Recencia,Frecuencia,Monetario
Nombre Cliente,,,
DISTRIBUIDORA PUNTO SEIS,-1.401963,-0.113701,-0.098525
5 ESQUINAS,0.147700,-0.138627,-0.134109
61PRADO EUROPEAN GUESTHOUSE,-1.401963,0.081548,0.029422
ABARROTES ALFER,-0.310155,-0.135857,-0.144276
ABARROTES CUCUTA,-0.098838,-0.133088,-0.147355
...,...,...,...
ZEUS,1.063409,-0.192632,-0.190568
ZONA LOGISTICA S.A.S,-1.437182,0.082933,0.133056
ZONA REFRESCANTE,1.063409,-0.192632,-0.195948


In [7]:
### MÉTODO DEL CODO (WCSS)

In [8]:
# Buscar el valor optimo de clusters:
wcss = {}
for k in range(2, len(rfm_scaler_df.index)):
    kmeans = KMeans(n_clusters=k, n_init=10, max_iter=300, random_state=42)
    kmeans.fit(rfm_scaler_df)
    wcss[k] = kmeans.inertia_
print(wcss.keys())

KeyboardInterrupt: 

In [9]:
# Grafica de codo:
plt.figure(figsize=(10, 6))
plt.plot(list(wcss.keys()), list(wcss.values()), o-) #Se usa o- para que se vea la linea y los puntos
plt.title('Grafica de codo para encontrar el número óptimo de clusters (K)')
plt.xlabel('Número de clusters (K)')
plt.ylabel('WCSS - Inercia')
plt.grid(True)
plt.show()

SyntaxError: invalid syntax (1321232338.py, line 3)

## *MÉTODO DE SILUETA*

In [ ]:
silhouette_scores = {}
for k in range(2, len(rfm_scaler_df.index)): # Buscar el valor optimo de clusters:
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, ramdom_state=42) # Inicializar el modelo KMeans
    kmeans.fit(rfm_scaler_df) # Ajustar el modelo KMeans
    score = silhouette_scores(rfm_scaler_df, kmeans.labels_) # Calcular el coeficiente de la silueta
    silhouette_scores[k] = score # Guardar el coeficiente de la silueta para cada k
    print(f"Coeficiente de la silueta para k={k}: {score:.4f}") # Imprimir el coeficiente de la silueta para cada k